In [1]:
from googlefinance.client import get_price_data, get_prices_data
import pandas as pd

In [22]:
# Single stock
# param_AAPL = {
#     'q': "AAPL", # Stock symbol (ex: "AAPL")
#     'i': "86400", # Interval size in seconds ("86400" = 1 day intervals)
#     'x': "NASD", # Stock exchange symbol on which stock is traded (ex: "NASDAQ, NYSE", "NYSEARCA")
#     'p': "10Y" # Period (Ex: "1Y" = 1 year) from today
# }
start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('2015-01-01')     
price = get_price_data({'q': "AAPL", 'i': "86400", 'x': "NASD", 'p': "20Y" })['Close'].loc[start : end]
price.index = pd.to_datetime(price.index.date)
#price = pd.DataFrame(price.values, columns=['Close'], index=price.index)
price.head()

2012-01-04    58.7500
2012-01-05    59.0600
2012-01-06    59.7200
2012-01-07    60.3400
2012-01-10    60.2471
Name: Close, dtype: float64

In [8]:
# Two stocks
stock1 = "XOM"
stock2 = "CVX"
start = pd.to_datetime('2003-01-01')
end =  pd.to_datetime('2009-01-01')
x = pd.DataFrame(
        get_price_data({'q': stock1, 'i': "86400", 'x': "NYSE", 'p': "10Y" }))['Close'].loc[start : end]
y =pd.DataFrame(
        get_price_data({'q': stock2, 'i': "86400", 'x': "NYSE", 'p': "10Y" }))['Close'].loc[start : end]
price = pd.concat([x, y], axis=1)
price.columns = [stock1, stock2]
price.index = pd.to_datetime(price.index.date)
price.head()

,XOM,CVX
2008-05-20,94.36,102.20
2008-05-21,94.56,103.09
2008-05-22,93.67,103.02
2008-05-23,92.51,101.91
2008-05-24,90.70,100.73


In [7]:
# Multiple stocks
start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('2014-12-31')

params_STOCKS = [{'q': "SBUX", 'x': "NASD",},
                              {'q': "AAPL", 'x': "NASD",},
                              {'q': "GS", 'x': "NYSE",},
                              {'q': "GILD", 'x': "NASD",}]
df = get_prices_data(params_STOCKS, "20Y")
df.index = pd.to_datetime(df.index)
df = df.loc[start:end]

df.head(2)

,SBUX_Open,SBUX_High,SBUX_Low,SBUX_Close,SBUX_Volume,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Volume,GS_Open,GS_High,GS_Low,GS_Close,GS_Volume,GILD_Open,GILD_High,GILD_Low,GILD_Close,GILD_Volume
2012-01-04,23.425,23.52,22.64,22.642,12923554.0,58.49,58.93,58.43,58.75,75564699.0,92.97,96.43,92.83,95.36,8646487.0,20.73,21.00,20.68,20.93,19564110.0
2012-01-05,22.700,23.32,22.64,23.080,13892148.0,58.57,59.24,58.47,59.06,65061108.0,94.73,95.13,93.68,94.74,7407211.0,20.98,21.03,20.85,21.01,16235940.0


In [21]:
start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('2014-12-31')     # Be cautious
df = get_prices_data( [{'q': ".INX", 'x': "INDEXSP",}], "20Y")     #{'q': "DAX", 'x': "INDEXDB",}
df.index = pd.to_datetime(df.index)
df = df['.INX_Close'].loc[start:end]
df.head()

2012-01-04    1277.06
2012-01-05    1277.30
2012-01-06    1281.06
2012-01-07    1277.81
2012-01-10    1280.70
Name: .INX_Close, dtype: float64

In [33]:
# params_INDEX = [
#     # S&P 500
#     {
#         'q': ".INX",
#         'x': "INDEXSP",
#     }
#]
def SPY_return(start, end):
    df = get_prices_data( [{'q': ".INX", 'x': "INDEXSP",}], "20Y")
    df.index = pd.to_datetime(df.index)
    df = pd.DataFrame(df['.INX_Close'].loc[start:end])
    df.rename(columns={".INX_Close":"SPY"}, inplace = True)
    return df

start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('2014-12-31')     # Be cautious
df = SPY_return(start, end)
df.head(2)

,Close
2012-01-04,1277.06
2012-01-05,1277.30


In [18]:
def multi_returns(start, end, symbols):
    columns = []
    for symbol, market in symbols:
        columns.append(symbol)
        
    df = pd.DataFrame(columns=columns)
    
    for symbol, market in symbols:
        df[symbol] = pd.DataFrame(get_price_data({'q': symbol, 'i': "86400", 'x': market, 'p': "20Y" }))['Close'].loc[start : end]
    
    df.index = pd.to_datetime(df.index.date)
    return df
        
start = pd.to_datetime('2016-01-01')
end = pd.to_datetime('2016-02-01')     
df = multi_returns(start, end, [("SBUX","NASD"), ("AAPL","NASD")])
df.pct_change().head()

,SBUX,AAPL
2016-01-01,NaN,NaN
2016-01-05,-0.029485,0.000855
2016-01-06,0.006694,-0.025059
2016-01-07,-0.008866,-0.019570
2016-01-08,-0.024772,-0.042205


In [ ]:
# Four asset example of the covariance matrix.
from googlefinance.client import get_price_data, get_prices_data
import pandas as pd

start = pd.to_datetime('2016-01-01')
end = pd.to_datetime('2016-02-01')     

df = pd.DataFrame(columns=['SBUX', 'AAPL', 'GS', 'GILD'])
df['SBUX'] = pd.DataFrame(get_price_data({'q': "SBUX", 'i': "86400", 'x': "NASD", 'p': "20Y" }))['Close'].loc[start : end]
df['AAPL'] = pd.DataFrame(get_price_data({'q': "AAPL", 'i': "86400", 'x': "NASD", 'p': "20Y" }))['Close'].loc[start : end]
df['GS'] = pd.DataFrame(get_price_data({'q': "GS", 'i': "86400", 'x': "NYSE", 'p': "20Y" }))['Close'].loc[start : end]
df['GILD'] = pd.DataFrame(get_price_data({'q': 'GILD', 'i': "86400", 'x': "NASD", 'p': "20Y" }))['Close'].loc[start : end]
df.index = pd.to_datetime(df.index.date)

returns = df.pct_change()[1:]

In [12]:
# Treasury Bill return
param = {
    'q': "BIL", # Use an ETF that tracks 3-month T-bills as our risk-free rate of return
    'i': "86400", # Interval size in seconds ("86400" = 1 day intervals)
    'x': "NYSEARCA", # Stock exchange symbol on which stock is traded (ex: "NASD")
    'p': "3Y" # Period (Ex: "1Y" = 1 year)
}

treasury_ret = get_price_data(param).pct_change()['Close'][1:]
treasury_ret.head()

2015-05-22 05:00:00    0.000219
2015-05-23 05:00:00   -0.000109
2015-05-27 05:00:00    0.000109
2015-05-28 05:00:00    0.000000
2015-05-29 05:00:00   -0.000219
Name: Close, dtype: float64

In [15]:
symbol_list = ['AAPL', 'ABC', 'ABBV', 'ACE', 'AGN', 'ADP', 'ADSK','ABT', 'AEE','ACN','AEP','ADBE','AET',
                      'ADI','ADM','ADS','AIV','ADT','ALL','AES','ALTR','ALXN','AFL','MAT','AIG','AMGN','AIZ']

import pandas as pd
import quandl
quandl.ApiConfig.api_key = 'U_Gp39gZutpHmaFgC65Y' 

start = '2014-12-01'
end = '2015-01-01'

historical_prices = pd.DataFrame()
for symbol in symbol_list:
    data = quandl.get('WIKI/' + symbol, start_date=start, end_date=end)['Adj. Close']
    historical_prices[symbol] = data
historical_prices.head()

,AAPL,ABC,ABBV,ACE,AGN,ADP,ADSK,ABT,AEE,ACN,...,ADT,ALL,AES,ALTR,ALXN,AFL,MAT,AIG,AMGN,AIZ
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-01,109.316981,86.746731,62.058252,111.439784,260.613665,80.240281,60.51,41.370450,38.414948,80.896454,...,32.936428,64.265052,12.162849,36.450021,192.51,55.436318,26.273529,51.587079,155.644096,63.475972
2014-12-02,108.898979,87.892659,62.201926,111.546500,261.743020,79.988569,60.74,42.210554,38.708396,80.877640,...,33.411100,64.740315,12.332149,36.715649,200.79,55.763679,26.282123,52.226823,158.328421,63.937787
2014-12-03,110.133985,87.844912,61.510494,112.225596,263.357801,80.119087,61.45,42.639940,38.992951,81.385595,...,33.662967,64.702294,12.305417,37.020628,198.88,55.838505,26.015692,52.150829,157.196702,64.691770
2014-12-04,109.715983,87.806714,62.471315,112.662159,265.002301,80.226297,61.09,42.462585,38.752858,81.216277,...,33.294854,65.006462,12.243044,36.892734,197.66,55.585969,26.368069,52.407309,155.775039,65.031063
2014-12-05,109.250481,88.054998,62.597030,112.603950,263.575746,80.007215,60.90,42.285230,38.494979,81.075178,...,33.730777,65.282115,12.189581,37.315770,198.61,55.660794,26.539960,52.559297,158.291009,64.833142
